# Watching videos in cloud storage bucket

In [191]:
import pandas as pd
from datetime import datetime as dt, timedelta as td
from IPython.display import clear_output as co

In [71]:
from modules.googlecloudstorage import GCS
from modules.image_similarity import load_nested_images_from_folder
import matplotlib.pyplot as plt
import cv2

---
### Download Cloud Storage bucket blobs to folder

In [59]:
cd ../

C:\Users\luisr\Desktop\Repositories\Data Science Projects\Hackaton COR IV - Centro de Operações do RJ\INCUBAÇÃO\Cameras


#### Import Google Cloud Storage wrapper module and set storage instance

In [75]:
from modules.googlecloudstorage import GCS

sa_json = '../../../../Apps/APIs/octa-api/credentials/octacity-iduff.json'
user_project = None
default_bucket_name = 'flood-video-collection'

gcs = GCS(sa_json, user_project, default_bucket_name)

#### List collection blobs with .mp4 extension (and file bytes size)

In [87]:
prefix = 'polygons/flood/1/313/'
delimiter = None
bucket_name = 'flood-video-collection'

ext = '.mp4'

drop_first = False
print_each = 1000

blobs = gcs.list_blobs(prefix, delimiter, bucket_name)

names = []
for i, blob in enumerate(blobs):
    if blob.name.endswith(ext):
        names.append([blob.name, blob.size])
    if (i + 1) % print_each == 0: print(f'\n- Blobs Searched: {i + 1}'); co(True)

print(f'\n- Blobs Searched: {i + 1}')
if drop_first: print(f'\n · First item excluded: {names[0]}'); del names[0]
print(f'\n · Results: {len(names)}')


- Blobs Searched: 31

 · Results: 31


#### Download blobs in blob to folder

In [91]:
folder = 'Dados/flood-video-collection/'
prefix = 'polygons/flood/1/313/'
delimiter = None
bucket_name = 'flood-video-collection'

gcs.download_to_folder(folder, prefix, delimiter, bucket_name, report_freq=5, overwrite=False)


DOWNLOAD FAILED. FILE ALREADY EXISTS. FILE: Dados/flood-video-collection/polygons/flood/1/313/CODE313 2023-02-11 20-50-00.mp4 · BLOB: polygons/flood/1/313/CODE313 2023-02-11 20:50:00.mp4 · BUCKET: flood-video-collection · (1/31)


---
## Exploratory data analysis - Cameras with top number of flood events

In [2]:
import pandas as pd, matplotlib.pyplot as plt, seaborn as sns; sns.set()

c:\Users\luisr\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


### Reload flood videos dataset

In [21]:
data_path = '../Dados'
control_path = f'{data_path}/Controle de vídeos/videos_control_19-04.csv'

control = pd.read_csv(control_path)
control['timestamp'] = pd.to_datetime(control['timestamp'])

control.columns

Index(['blob_name', 'blob_size', 'bucket_name', 'file_name', 'code',
       'n_folders', 'timestamp', 'folder_structure'],
      dtype='object')

#### Get event types

In [36]:
def get_event_info(blob_name):
    "returns event type and event id given `blob_name` matching the folder structure `{source}/{type}/{event}/{code}`."
    info = blob_name.split('/')
    if len(info) != 5 or info[0] == 'polygons': return ['', '']
    return ['/'.join(info[:2]), info[2]]

#### Events recorded per câmera

In [54]:
# folder_structure_msk = control['folder_structure'] =='{source}/{type}/{event}/{code}'

event_info = control['blob_name'].apply(get_event_info).tolist()
control[['event_type', 'event_id']] = event_info

control_comando = control[control['event_type'].isin(['comando/bolsão', 'comando/alagamento'])]
events_per_code = control_comando.groupby('code')['event_id'].nunique()

cams_recurrence = events_per_code[events_per_code > 1]
display(cams_recurrence.sort_values(ascending=False).to_frame('Sistema Comando Events recorded').T)

print('Number of cameras with more than one event in Sistema Comando database:', len(cams_recurrence))

code,38,1671,2015,1880,18,226,249,278,1726,1881,2165,2166,2167,2219,2220,2221
Sistema Comando Events recorded,5,5,5,3,2,2,2,2,2,2,2,2,2,2,2,2


Number of cameras with more than one event in Sistema Comando database: 16


---
## Watch videos from folder

#### Add clock to video inplace

In [218]:
os.makedirs('test/test')

In [222]:
import os
from datetime import timedelta

def round_seconds(tm):
    return tm - timedelta(seconds=tm.second, microseconds=tm.microsecond)

def add_text_to_video(frame, text):
    cv2.putText(frame, text, org=(550, 27), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
    fontScale=0.67, color=(255, 255, 255), thickness=2, lineType=cv2.LINE_AA)
    return frame

def add_clock_to_video_inplace(folder, path, to_folder, fps:int=3, shape:tuple=(854, 480), codec:str='mp4v'):
    """
    path: path to video file containing a timestamp in th file name
    """
    file_name = path.split('/')[-1]
    stamp = ' '.join(file_name.split(' ')[1:])
    timestamp = dt.strptime(stamp, '%Y-%m-%d %H-%M-%S.mp4')[:]
    offset = pd.offsets.Second() / fps
    full_path = f'{folder}/{path}'
    to_full_path = f'{to_folder}/{path}'
    full_folder = '/'.join(to_full_path.split('/')[:-1])
    if not os.path.exists(full_folder): os.makedirs(full_folder)
    cap = cv2.VideoCapture(full_path)
    video = cv2.VideoWriter(to_full_path, cv2.VideoWriter_fourcc(*codec), fps, shape)
    while cap.isOpened():
        r, frame = cap.read()
        if not r:
            break
        frame = add_text_to_video(frame, str(timestamp))
        timestamp += offset
        video.write(frame)
    video.release()

#### Concatenate with opencv

In [109]:
def concatenate_videos_from_folder(path, folder, ext:str='.mp4', fps:int=9, shape:tuple=(854, 480), codec:str='mp4v',):
    """
    folder: string with path to folder containing video files with extension `ext` 
    """
    video = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*codec), fps, shape)
    for file in os.listdir(folder):
        if file.endswith(ext):
            cap = cv2.VideoCapture(folder + file)
            while cap.isOpened():
                r, frame = cap.read()
                if not r:
                    break
                video.write(frame)            
    video.release()

#### Concatenate videos with moviepy library

In [94]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

In [96]:
import os

path = folder+prefix
files = [path + file for file in os.listdir(path)]

clip_1 = VideoFileClip(files[0])
clip_2 = VideoFileClip(files[1])
final_clip = concatenate_videoclips([clip_1,clip_2])
final_clip.write_videofile("final.mp4")

t:   0%|                                                                              | 0/45 [00:00<?, ?it/s, now=None]

Moviepy - Building video final.mp4.
Moviepy - Writing video final.mp4



Moviepy - Done !
Moviepy - video ready final.mp4


## Testing concat and timestamp functions

### Concatenate videos from folder

In [111]:
from time import time

path = 'final.mp4'
folder = f'Dados/{bucket_name}/{prefix}'

s = time()
concatenate_videos_from_folder(path, folder, fps=9)
t = time() - s

t

### Add clock to video inplace

In [223]:
path = 'polygons/flood/1/313/CODE313 2023-02-11 20-50-00.mp4'

folder, to_folder = 'Dados/flood-video-collection', 'Dados/flood-video-collection-stamped'

add_clock_to_video_inplace(folder, path, to_folder, fps=3, shape=(854, 480), codec='mp4v')

In [215]:
folder, to_folder

('Dados/flood-video-collection', 'Dados/flood-video-collection-stamped')